# LNG

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.models.regression import MotionRegression, Regression

from vessel_manoeuvring_models.parameters import df_parameters
from vessel_manoeuvring_models.substitute_dynamic_symbols import run
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from vessel_manoeuvring_models.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from vessel_manoeuvring_models.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust

from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import initial_state, extended_kalman_filter, extended_kalman_smoother, guess_covariance_matrixes
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler


In [ ]:
%reload_kedro

ship='LNG'
id = "17902"
vmm_name = "vmm_martin"

dataframes = {

"raw_data" : catalog.load(f"{ship}.{id}.data"),   
"smooth2" : catalog.load(f"{ship}.updated.{id}.data_ek_smooth"),   
    
#"simulation" : catalog.load(f"{ship}.updated.{vmm_name}.joined.{id}.data_resimulate"),   

}
dataframes['raw_data']['psi_deg'] = np.rad2deg(dataframes['raw_data']['psi'])

ship_data = catalog.load(f"{ship}.ship_data")
model = catalog.load(f"{ship}.updated.{vmm_name}.joined.model")
model.parameters.Xvr=0
model.parameters.Nu=0
#model.parameters.Nrdeltadelta = 0

ek = catalog.load(f"{ship}.{vmm_name}.ek")



In [ ]:
model.parameters

In [ ]:
df_sim = simulate_euler(data=dataframes['smooth2'], model=model, ek=ek, solver='euler')
dataframes['simulation2'] = df_sim

In [ ]:
track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'], time_window=[0,20])
#track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'])

In [ ]:
plot(dataframes=dataframes, keys=['u','v','r','delta','psi_deg'], time_window=[0,20]);
#plot(dataframes=dataframes, keys=['u','v','r','delta','psi_deg']);

In [ ]:
model.parameters

In [ ]:
np.rad2deg(1.492609)

In [ ]:
dataframes['simulation'].loc[16.50:]